## BLE RSSI Dataset for Indoor localization

The dataset was created using the RSSI readings of an array of 13 ibeacons in the first floor of Waldo Library, Western Michigan University. Data was collected using iPhone 6S. The dataset contains two sub-datasets: a labeled dataset (1420 instances) and an unlabeled dataset (5191 instances). The recording was performed during the operational hours of the library. For the labeled dataset, the input data contains the location (label column), a timestamp, followed by RSSI readings of 13 iBeacons. RSSI measurements are negative values. Bigger RSSI values indicate closer proximity to a given iBeacon (e.g., RSSI of -65 represent a closer distance to a given iBeacon compared to RSSI of -85). For out-of-range iBeacons, the RSSI is indicated by -200. The locations related to RSSI readings are combined in one column consisting a letter for the column and a number for the row of the position. The following figure depicts the layout of the iBeacons as well as the arrange of locations.

# Configure docker credentials

Get your docker registry user and password encoded in base64 

echo -n USER:PASSWORD | base64 

Create a config.json file with your Docker registry url and the previous generated base64 string 

In [1]:
!echo -n USER:PASSWORD | base64

VVNFUjpQQVNTV09SRA==


In [2]:
%%writefile config.json
{
    "auths": {
        "https://index.docker.io/v1/": {
            "auth": "<<Provide previous generated base64 string>>"
        }
    }
}

Writing config.json


## Create requirements.txt

In [3]:
%%writefile requirements.txt
pandas
joblib
numpy
scikit-learn>=0.21.0,<0.22
seldon-core
tornado>=6.0.3
tensorflow==1.13.1
keras==2.2.4
google-cloud-storage
kubeflow-tfjob
azure==4.0.0
kubeflow-fairing
kubernetes==10.0.1

Writing requirements.txt


## Install the packages listed in requirements.txt using pip

In [4]:
!pip install --user -r requirements.txt

You are using pip version 19.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Import Libraries

In [5]:
from kubernetes.client import V1PodTemplateSpec
from kubernetes.client import V1ObjectMeta
from kubernetes.client import V1PodSpec
from kubernetes.client import V1Container
from kubernetes.client import V1VolumeMount
from kubernetes.client import V1Volume
from kubernetes.client import V1PersistentVolumeClaimVolumeSource

from kubeflow.tfjob import constants
from kubeflow.tfjob import utils
from kubeflow.tfjob import V1ReplicaSpec
from kubeflow.tfjob import V1TFJob
from kubeflow.tfjob import V1TFJobSpec
from kubeflow.tfjob import TFJobClient

import time
import re, os
import tensorflow as tf
import logging
import sys
import importlib

tfjob_client = TFJobClient()
namespace = utils.get_default_target_namespace()
print("Namespace : %s"%namespace)

/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Namespace : anonymous


## Set up Kubeflow Fairing for training and predictions on On-premise
Import the fairing library and configure the onprem environment that your training or prediction job will run in.

In [6]:
from kubernetes import client as k8s_client
from kubernetes.client import rest as k8s_rest
from kubernetes import config as k8s_config
from kubernetes.client.rest import ApiException

from kubeflow import fairing   
from kubeflow.fairing import utils as fairing_utils
from kubeflow.fairing import TrainJob
from kubeflow.fairing.preprocessors.function import FunctionPreProcessor
from kubeflow.fairing.preprocessors import base as base_preprocessor
from kubeflow.fairing.builders.cluster.cluster import ClusterBuilder

from kubeflow.fairing.cloud.k8s import MinioUploader
from kubeflow.fairing.builders.cluster.minio_context import MinioContextSource
from kubeflow.fairing import PredictionEndpoint
from kubeflow.fairing.kubernetes.utils import mounting_pvc
from kubeflow.fairing.kubernetes.utils import mounting_pvc

## Get minio-service cluster IP to upload docker build context
#### Set DOCKER_REGISTRY
The DOCKER_REGISTRY variable is used to push the newly built image. 
Please change the variable to the registry for which you've configured credentials.

In [7]:
PY_VERSION = ".".join([str(x) for x in sys.version_info[0:3]])
BASE_IMAGE = 'python:{}'.format(PY_VERSION)
DOCKER_REGISTRY = "samba07"

k8s_config.load_incluster_config()
namespace = fairing_utils.get_current_k8s_namespace()
print(namespace)

api_client = k8s_client.CoreV1Api()
minio_service_endpoint = None

try:
    minio_service_endpoint = api_client.read_namespaced_service(name='minio-service', namespace='kubeflow').spec.cluster_ip
except ApiException as e:
    if e.status == 403:
        logging.warning(f"The service account doesn't have sufficient privileges "
                      f"to get the kubeflow minio-service. "
                      f"You will have to manually enter the minio cluster-ip. "
                      f"To make this function work ask someone with cluster "
                      f"priveleges to create an appropriate "
                      f"clusterrolebinding by running a command.\n"
                      f"kubectl create --namespace=kubeflow rolebinding "
                       "--clusterrole=kubeflow-view "
                       "--serviceaccount=${NAMESPACE}:default-editor "
                       "${NAMESPACE}-minio-view")
        logging.error("API access denied with reason: {e.reason}")

s3_endpoint = minio_service_endpoint
minio_endpoint = "http://"+s3_endpoint+":9000"
minio_username = "minio"
minio_key = "minio123"
minio_region = "us-east-1"
print(minio_endpoint)


minio_uploader = MinioUploader(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)
minio_context_source = MinioContextSource(endpoint_url=minio_endpoint, minio_secret=minio_username, minio_secret_key=minio_key, region_name=minio_region)

anonymous
http://10.106.203.39:9000


### Create a config-map in the namespace you're using with the docker config

In [8]:
!kubectl create --namespace $namespace configmap docker-config --from-file=./config.json

configmap/docker-config created


## Build docker image

Note: Upload dataset, Dockerfile, and blerssi-model.py into notebook

In [9]:
#output_map is a map of extra files to add to the notebook.
# It is a map from source location to the location inside the context.
output_map= {
    "Dockerfile": "Dockerfile", #Dockerfile
    "blerssi-model.py":"blerssi-model.py",
    "iBeacon_RSSI_Labeled.csv": "iBeacon_RSSI_Labeled.csv"
}
preprocessor = base_preprocessor.BasePreProcessor(
    input_files=['iBeacon_RSSI_Labeled.csv', "requirements.txt"],
    output_map=output_map)

preprocessor.preprocess()
builder = ClusterBuilder(registry=DOCKER_REGISTRY, base_image=BASE_IMAGE, preprocessor=preprocessor,
                        context_source=minio_context_source)

builder.build()

Building image using cluster builder.
Creating docker context: /tmp/fairing_context_j0cvmil9
/tmp/fairing_dockerfile_964yplok already exists in Fairing context, skipping...
Waiting for fairing-builder-6pjzz-zb7nr to start...
Waiting for fairing-builder-6pjzz-zb7nr to start...
Waiting for fairing-builder-6pjzz-zb7nr to start...
Waiting for fairing-builder-6pjzz-zb7nr to start...
Pod started running True


INFO[0001] Resolved base name tensorflow/tensorflow:1.7.0-py3 to tensorflow/tensorflow:1.7.0-py3
INFO[0001] Resolved base name tensorflow/tensorflow:1.7.0-py3 to tensorflow/tensorflow:1.7.0-py3
INFO[0001] Downloading base image tensorflow/tensorflow:1.7.0-py3
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:7e3da1ea0cd5150dcb8412b35f4baa59c3105ef9aebf77cb04204dcdde560331: no such file or directory
INFO[0002] Downloading base image tensorflow/tensorflow:1.7.0-py3
INFO[0002] Built cross stage deps: map[]
INFO[0002] Downloading base image tensorflow/tensorflow:1.7.0-py3
INFO[0003] Error while retrieving image from cache: getting file info: stat /cache/sha256:7e3da1ea0cd5150dcb8412b35f4baa59c3105ef9aebf77cb04204dcdde560331: no such file or directory
INFO[0003] Downloading base image tensorflow/tensorflow:1.7.0-py3
INFO[0004] Unpacking rootfs as cmd RUN chmod +x /opt/blerssi-model.py  /opt/iBeacon_RSSI_Labeled.csv requires it.
INFO[0022] Taking snaps

In [10]:
builder.image_tag

'samba07/fairing-job:8A51B26'

## Define TFJob Class to create training job

In [11]:
class Tfjob(object):

    def get_tfjob_params(self):
    
        #Defining a Volume Mount
        volume_mount = V1VolumeMount(name="nfsvolume", mount_path="/mnt/")

        #Defining a Persistent Volume Claim
        persistent_vol_claim = V1PersistentVolumeClaimVolumeSource(claim_name="nfs1")

        #Defining a Volume
        volume = V1Volume(name="nfsvolume", persistent_volume_claim=persistent_vol_claim)
        
        #Defining a Container
        container = V1Container(
            name="tensorflow",            
            image=builder.image_tag,
            volume_mounts=[volume_mount]
        )
        
        return (volume_mount, persistent_vol_claim, volume, container)
        
    def get_tfjob_nodes(self):
    
        params = self.get_tfjob_params()

        #Defining a Master
        master = V1ReplicaSpec(replicas=1,
                               restart_policy="Never",
                               template=V1PodTemplateSpec(spec=V1PodSpec(
                                                    containers=[params[3]],
                                                    volumes=[params[2]])))
        
        #Defining Worker Spec
        worker = V1ReplicaSpec(replicas=1,
                               restart_policy="Never",
                               template=V1PodTemplateSpec(spec=V1PodSpec(
                                                    containers=[params[3]],
                                                    volumes=[params[2]])))
        
        #Defining Parameter server(PS) Spec
        ps = V1ReplicaSpec(replicas=1,
                               restart_policy="Never",
                               template=V1PodTemplateSpec(spec=V1PodSpec(
                                                    containers=[params[3]],
                                                    volumes=[params[2]])))
        
        return (master,worker,ps)
    
    def create_tfjob(self):
        global tfjob_name
        tfjob_name="blerssi-tfjob-%s"%int(time.time())
        print("TFJob name : %s " %tfjob_name)
        tfjob_node_spec = self.get_tfjob_nodes()
        
        #Defining TFJob
        tfjob = V1TFJob(
            api_version="kubeflow.org/v1",
            kind="TFJob",
            metadata=V1ObjectMeta(name=tfjob_name,namespace=namespace),
            spec=V1TFJobSpec(
                clean_pod_policy="None",
                tf_replica_specs={"PS":tfjob_node_spec[2],"Worker": tfjob_node_spec[1],"Master":tfjob_node_spec[0]}
            )
        )
        
        #Creating TFJob
        tfjob_client = TFJobClient()
        tfjob_client.create(tfjob, namespace=namespace)       

## Define Blerssi class to be used by Kubeflow fairing 
## ( Must necessarily contain train() and predict() methods)

In [12]:
class BlerssiServe(object):
    
    def __init__(self):
        self.model=None
        
    def train(self):
        
        Tfjob().create_tfjob()
        
    def predict(self,X,feature_names=None):
        
        feature_col=["b3001", "b3002","b3003","b3004","b3005","b3006","b3007","b3008","b3009","b3010","b3011","b3012","b3013"]
        model_input1=tf.train.Example()
        for i in range(len(X)):
            model_input1.features.feature[feature_col[i]].float_list.value.append(X[i])
            
        path=os.path.join(os.getcwd(), "/mnt")
        for dir in os.listdir(path):
            if re.match('[0-9]',dir):
                exported_path=os.path.join(path,dir)
                break
        
        # Open a Session to predict
        with tf.Session() as sess:
         tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], exported_path)
         model_input =model_input1
         
         predictor= tf.contrib.predictor.from_saved_model(exported_path,signature_def_key='predict')
         input_tensor=tf.get_default_graph().get_tensor_by_name("input_example_tensor:0")
            
         model_input=model_input.SerializeToString()
         output_dict= predictor({"examples":[model_input]})
        sess.close()
        
        response = output_dict.items()
        print(response)
        response1 = output_dict['class_ids']
        return response1

## Train an Blerssi model remotely on Kubeflow
Kubeflow Fairing packages the BlerssiServe class, the training data, and the training job's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the training job on kubeflow.

In [13]:
BackendClass = getattr(importlib.import_module('kubeflow.fairing.backends'), "KubernetesBackend")
train_job = TrainJob(BlerssiServe, input_files=["iBeacon_RSSI_Labeled.csv", "requirements.txt"],
                     pod_spec_mutators = [mounting_pvc(pvc_name="nfs1", pvc_mount_path="/mnt/")],
                     docker_registry=DOCKER_REGISTRY, base_docker_image=BASE_IMAGE, backend=BackendClass(build_context_source=minio_context_source))
train_job.submit()

Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/root/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_00igasbe
/root/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-4w5gh-dpnmg to start...
Waiting for fairing-builder-4w5gh-dpnmg to start...
Waiting for fairing-builder-4w5gh-dpnmg to start...
Waiting for fairing-builder-4w5gh-dpnmg to start...
Pod started running True


INFO[0001] Resolved base name python:3.6.5 to python:3.6.5
INFO[0001] Resolved base name python:3.6.5 to python:3.6.5
INFO[0001] Downloading base image python:3.6.5
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:bf5470ee0ee924cf7f70cfe3087abd0fb0a4d5c4f3e9f3624fd4950b54cf6549: no such file or directory
INFO[0002] Downloading base image python:3.6.5
INFO[0003] Built cross stage deps: map[]
INFO[0003] Downloading base image python:3.6.5
INFO[0004] Error while retrieving image from cache: getting file info: stat /cache/sha256:bf5470ee0ee924cf7f70cfe3087abd0fb0a4d5c4f3e9f3624fd4950b54cf6549: no such file or directory
INFO[0004] Downloading base image python:3.6.5
INFO[0004] Executing 0 build triggers
INFO[0004] Unpacking rootfs as cmd COPY /app//requirements.txt /app/ requires it.
INFO[0019] Taking snapshot of full filesystem...
INFO[0025] WORKDIR /app/
INFO[0025] cmd: workdir
INFO[0025] Changed working directory to /app/
INFO[0025] Creating direc

The job fairing-job-548fp launched.
Waiting for fairing-job-548fp-gclbd to start...
Waiting for fairing-job-548fp-gclbd to start...
Waiting for fairing-job-548fp-gclbd to start...
Waiting for fairing-job-548fp-gclbd to start...
Pod started running True


/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Cleaning up job fairing-job-548fp...


'fairing-job-548fp'

## Deploy the trained model to Kubeflow for predictions
Kubeflow Fairing packages the BlerssiServe class, the trained model, and the prediction endpoint's software prerequisites as a Docker image. Then Kubeflow Fairing deploys and runs the prediction endpoint on Kubeflow.

In [14]:
endpoint = PredictionEndpoint(BlerssiServe, input_files=["requirements.txt"],
                              docker_registry=DOCKER_REGISTRY,
                              pod_spec_mutators = [mounting_pvc(pvc_name="nfs1", pvc_mount_path="/mnt/")],
                              backend=BackendClass(build_context_source=minio_context_source))
endpoint.create()

Using default base docker image: registry.hub.docker.com/library/python:3.6.5
Using builder: <class 'kubeflow.fairing.builders.cluster.cluster.ClusterBuilder'>
Building the docker image.
Building image using cluster builder.
/root/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Creating docker context: /tmp/fairing_context_icugl3mn
/root/.local/lib/python3.6/site-packages/kubeflow/fairing/__init__.py already exists in Fairing context, skipping...
Waiting for fairing-builder-znckp-h2c6m to start...
Waiting for fairing-builder-znckp-h2c6m to start...
Waiting for fairing-builder-znckp-h2c6m to start...
Waiting for fairing-builder-znckp-h2c6m to start...
Pod started running True


INFO[0001] Resolved base name registry.hub.docker.com/library/python:3.6.5 to registry.hub.docker.com/library/python:3.6.5
INFO[0001] Resolved base name registry.hub.docker.com/library/python:3.6.5 to registry.hub.docker.com/library/python:3.6.5
INFO[0001] Downloading base image registry.hub.docker.com/library/python:3.6.5
INFO[0002] Error while retrieving image from cache: getting file info: stat /cache/sha256:bf5470ee0ee924cf7f70cfe3087abd0fb0a4d5c4f3e9f3624fd4950b54cf6549: no such file or directory
INFO[0002] Downloading base image registry.hub.docker.com/library/python:3.6.5
INFO[0003] Built cross stage deps: map[]
INFO[0003] Downloading base image registry.hub.docker.com/library/python:3.6.5
INFO[0004] Error while retrieving image from cache: getting file info: stat /cache/sha256:bf5470ee0ee924cf7f70cfe3087abd0fb0a4d5c4f3e9f3624fd4950b54cf6549: no such file or directory
INFO[0004] Downloading base image registry.hub.docker.com/library/python:3.6.5
INFO[0004] Executing 0 build trig

Deploying the endpoint.
Cluster endpoint: http://fairing-service-6sj9t.anonymous.svc.cluster.local:5000/predict
Prediction endpoint: http://fairing-service-6sj9t.anonymous.svc.cluster.local:5000/predict


## Wait for  prediction pod ready state

In [15]:
!kubectl get deploy -l fairing-deployer=serving -n anonymous

NAME                     READY   UP-TO-DATE   AVAILABLE   AGE
fairing-deployer-wj5cp   1/1     1            1           3m41s


##  Get prediction endpoint

In [16]:
endpoint.url

'http://fairing-service-6sj9t.anonymous.svc.cluster.local:5000/predict'

## Call the prediction endpoint
Use the endpoint from previous cell

In [18]:
! curl -v http://fairing-service-6sj9t.anonymous.svc.cluster.local:5000/predict -H "Content-Type: application/x-www-form-urlencoded" -d 'json={"data":{"ndarray":[-0.458086, -0.6244, 2.354243, -0.404581, 1.421444, 1.767642, 2.637829, -0.603085, 0.382779, -0.378999, -0.341798, -0.303249, -0.327776]}}'

*   Trying 10.109.147.3...
* TCP_NODELAY set
* Connected to fairing-service-6sj9t.anonymous.svc.cluster.local (10.109.147.3) port 5000 (#0)
> POST /predict HTTP/1.1
> Host: fairing-service-6sj9t.anonymous.svc.cluster.local:5000
> User-Agent: curl/7.58.0
> Accept: */*
> Content-Type: application/x-www-form-urlencoded
> Content-Length: 162
> 
* upload completely sent off: 162 out of 162 bytes
* HTTP 1.0, assume close after body
< HTTP/1.0 200 OK
< Content-Type: application/json
< Content-Length: 54
< Access-Control-Allow-Origin: *
< Server: Werkzeug/1.0.1 Python/3.6.5
< Date: Thu, 09 Apr 2020 14:39:34 GMT
< 
{"data":{"names":["t:0"],"ndarray":[[70]]},"meta":{}}
* Closing connection 0


## Clean up the prediction endpoint
Delete the prediction endpoint created by this notebook.

In [19]:
endpoint.delete()

Deleting the endpoint. 
Deleted service: anonymous/fairing-service-6sj9t
Deleted deployment: anonymous/fairing-deployer-wj5cp
